### EET to Neet data preparation

In [1]:
# Load the data for analysis
#All import go here
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

#Get and describe data
path = '/home/workspace/files/aryan/data_files_raw/diversity_changes/Economic Status.csv'
df = pd.read_csv(path)
df = df.sort_values(['Client Number'])
#df = df.drop(['Minutes Total','Num Sessions','Num Weeks','Minutes per week'],1)
#df.rename(columns = {'Do ':'Full Assessment'}, inplace = True)
df
###################################
# Fill the numeric values with nan with 0
#df['TotalDisabilty'] = df['TotalDisabilty'].fillna(0)
#df['TotalMentalHealth'] = df['TotalMentalHealth'].fillna(0)

Results here


In [2]:
# Take the data from Economic Codes
# Merge them with the data of the clients 
path = '/home/workspace/files/aryan/data_files_raw/diversity_changes/Economic Codes.csv'
df1 = pd.read_csv(path)
df = pd.merge(df,df1,how="left",
                      left_on='Economic Status',
              right_on='Description',
                      left_index=False,
                      right_index=False,
                      sort=False,
                      suffixes=("_x", "_y"),
                      copy=True,
                      indicator=False,
                      validate=None,
                     )
df=df.drop(['Code','Economic Status','EET Detail','Description'],1)
df

Results here !!


In [3]:
# Now club them with the client numbers
df['Num Entries'] = df.groupby('Client Number')['Client Number'].transform('count')
df = df[df['Num Entries']>1]
df = df[df['EET status']!='Not Known']
df = df.sort_values(['Client Number','Until'])
df.head(40)

Results here !!


In [4]:
# Take out teh final result of the neet or the eet status
df_final = df.drop_duplicates(subset=['Client Number'], keep='last')

In [4]:
# Now drop the matching columns to make sure that the last occurences are reoveed
df_upper_occurences = pd.concat([df, df_final])
df_upper_occurences = df_upper_occurences.drop_duplicates(keep=False)
df_upper_occurences

Results here !


In [6]:
# Now find if teh NEET exists in the resulting upper occurences
# The Existsg column will containt the occurences of the NEET in the finbal column 
# Neet presence will give it a True or it will a false
g = df_upper_occurences.groupby('Client Number')
df_upper_occurences['Exists'] = g['EET status'].transform(lambda x: 'NEET' if (x.eq('NEET').any()) else 'EET' )

In [9]:
#df_upper_occurences.replace(True,'NEET', inplace=True)
#df_upper_occurences.replace(False,'EET', inplace = True)
df_initial = df_upper_occurences.drop_duplicates(subset=['Client Number'], keep='first')

# Droping the other rows from the database
# keep the only intial rows of teh data 
df_intial = df_initial.drop(['Until','EET status','Num Entries'],1)

In [10]:
df_final = df_final.drop(['Until','Num Entries'],1)
df_final = df_final.rename(columns={'EET status':'Final'})

In [5]:
df = pd.merge(df_initial,df_final,how="left",
                      on='Client Number',
                      left_index=False,
                      right_index=False,
                      sort=False,
                      suffixes=("_x", "_y"),
                      copy=True,
                      indicator=False,
                      validate=None,
                     )
df

Data here!


In [6]:
df = df.drop(['Until','EET status','Num Entries'],1)
df = df.rename(columns={'Exists':'Initial'})
df.head(3)

Data here !


In [7]:
### Now just use this to determine the FInal Outcome 
# Neet -> eet      +ve
# eet -> eet       +ve
# eet -> neet      -ve
# neet -> neet     -ve

df1 = df.copy(deep=True)

def result(row):
    if (row.Initial=='NEET') and(row.Final=='EET') :
        return "Positive"
    elif (row.Initial=='EET') and(row.Final=='EET'):
        return "Positive"
    else:
        return "Negative"

df1 = df1.assign(Result=df1.apply(result, axis=1))

df1.head(50)

Results here !1


In [14]:
df1['Result'].value_counts()

Negative    824
Positive    413
Name: Result, dtype: int64

In [18]:
#Just take the initial part that is oign to be used in tthe Development of the model
df1 = df1.drop(['Final'],1)
df1.to_csv('/home/workspace/files/aryan/data_files_raw/diversity_changes/eet_neet_outputs.csv', index=False)

## Check the Max rows we can get from different datasets

In [8]:
#Loading teh diversity dataset
path = '/home/workspace/files/aryan/data_files_raw/diversity_changes/eet_neet_outputs.csv'
eet_neet = pd.read_csv(path)
eet_neet = eet_neet.sort_values(by=['Client Number'])
print(eet_neet.shape)
eet_neet.head(2)

Results here !


In [9]:
#Loading teh diversity dataset
path = '/home/workspace/files/stbasil_prj/org_data/YP Background_with_MH_Disability.csv'
yp_back1 = pd.read_csv(path)
yp_back1 = yp_back1.sort_values(by=['Client Number'])
yp_back1 = yp_back1[['Client Number','TotalDisabilty','TotalMentalHealth']]
yp_back1.head(2)

Results here !


In [10]:
# Merge the medication and the alergies with the dat 
path = '/home/workspace/files/aryan/data_files_raw/yp_background.csv'
yp_back = pd.read_csv(path)
yp_back = yp_back.sort_values(by=['Client Number'])
yp_back = pd.merge(yp_back,yp_back1,how="left",
                      on='Client Number',
                      left_index=False,
                      right_index=False,
                      sort=False,
                      suffixes=("_x", "_y"),
                      copy=True,
                      indicator=False,
                      validate=None,
                     )
print(yp_back.shape)
yp_back.head(2)

Results here !


### check with the Yp Backgorund

In [11]:
df = pd.merge(eet_neet,yp_back,how="left",
                      on='Client Number',
                      left_index=False,
                      right_index=False,
                      sort=False,
                      suffixes=("_x", "_y"),
                      copy=True,
                      indicator=False,
                      validate=None,
                     )
#df= df[df['Result'].notnull()]
print(df.shape)
df.head(2)

data here


## Check with Diversity dataset

In [12]:
# Merge the medication and the alergies with the dat 
path = '/home/workspace/files/aryan/data_files_raw/diversity_changes/merged_without_null_rows.csv'
merged = pd.read_csv(path)
print(merged.shape)
merged = merged.drop_duplicates(subset=['Client Number'], keep='first')
df1 = pd.merge(df,merged,how="left",on='Client Number')
print(df1.shape)
df1.head(2)

Results here


## Check with Support hrs dataset

In [13]:
#Now load and check the Support hrs dataset
path = '/home/workspace/files/aryan/data_files_raw/Support_modified.csv'
support_hrs = pd.read_csv(path)
print(support_hrs.shape)
support_hrs.head(2)

Reuslts here


In [14]:
df = pd.merge(df1,support_hrs,how="left",
                      left_on='Client Number',
              right_on='Client No',
                      left_index=False,
                      right_index=False,
                      sort=False,
                      suffixes=("_x", "_y"),
                      copy=True,
                      indicator=False,
                      validate=None,
                     )
#df= df[df['Result'].notnull()]
print(df.shape)
df.head(2)

results here!


## Check withTenants dataset

In [15]:
#Now load and check the Support hrs dataset
path = '/home/workspace/files/aryan/data_files_raw/Tenants.csv'
tenants = pd.read_csv(path)
print(tenants.shape)
tenants = tenants.drop_duplicates(subset=['Client Number'], keep='last')
print(tenants.shape)
tenants.head(2)

Results here !


In [16]:
df = pd.merge(df,tenants,how="left",
                      on='Client Number',
                      left_index=False,
                      right_index=False,
                      sort=False,
                      suffixes=("_x", "_y"),
                      copy=True,
                      indicator=False,
                      validate=None,
                     )
#df= df[df['Result'].notnull()]
print(df.shape)
df.head(2)

Results here 


## Check for Life skills program

In [17]:
#Now load and check the Support hrs dataset
path = '/home/workspace/files/aryan/data_files_raw/Life Skills.csv'
life = pd.read_csv(path)
print(life.shape)
life = life.drop_duplicates(subset=['Client Number'], keep='last')
print(life.shape)
life.head(2)

Results here 


In [18]:
df = pd.merge(df,life,how="left",
                      on='Client Number',
                      left_index=False,
                      right_index=False,
                      sort=False,
                      suffixes=("_x", "_y"),
                      copy=True,
                      indicator=False,
                      validate=None,
                     )
#df= df[df['Result'].notnull()]
print(df.shape)
df.head(2)

Results here 


### Check for whole

In [30]:
df= df.drop(['Salary/Income', 'Immigration Status/Eligibility Reason','Please List any Medications/Allergies',
       'Communication/Language Barriers',
       'Mental Health/Learning Disabilities', 
       'If current accommodation is not the main applicants last settled home, Please describe the accommodation when last settled:',
        'Disabilities or Health Needs',
       'The YP is Vulnerable because of:',
       'Full Assessment of Eligibility',
       'Before approaching St Basils, did the YP contact any of the following for help', 'Code',
       'Description', 'EET Detail', 'Client No','Minutes Total','S_Start'],1)

In [31]:
df.rename(columns={'Does the YP have a local Connection':'Local Connection','Accommodation Type (Tenure at time of application)':'Prev Accomodation'},inplace=True)

In [32]:
#Now select the variables which are of most importance beacuse of their computation
features_with_less_nan = []
for feature in df.columns:
    nan_percentage = np.round(df[feature].isnull().mean(), 4)
    if(nan_percentage<0.5):
        features_with_less_nan.append(feature)

df1 = df[features_with_less_nan]
print(df1.shape)
print("R=The distribution of classes is :")
print(df1['Result'].value_counts())
df1.columns

(1237, 46)
R=The distribution of classes is :
Negative    824
Positive    413
Name: Result, dtype: int64


Index(['Client Number', 'Initial', 'Result', 'Gender', 'Preferred Language',
       'Nationality', 'Do They Have Any Medical Issues',
       'Is An Interpreter required', 'Local Connection', 'Prev Accomodation',
       'A1', 'A2', 'B1', 'B2', 'C1', 'D1', 'E1', 'E2', 'F1', 'F2', 'F3', 'G1',
       'H1', 'H3', 'I1', 'J1', 'K1', 'L1', 'TotalDisabilty',
       'TotalMentalHealth', 'Marital Status', 'Sexual Orientation', 'Religion',
       'Economic Status', 'EET status', 'Num Sessions', 'Time per Session',
       'Minutes per week', 'Num Weeks', 'Ref_CN', 'Ref No', 'Service Type',
       'Status', 'Area', 'Age at Start', 'Scheme'],
      dtype='object')

In [19]:
from matplotlib import pyplot as plt
def plot_nas(df: pd.DataFrame):
    if df.isnull().sum().sum() != 0:
        na_df = (df.notnull().sum() / len(df)) * 100      
        na_df = na_df.drop(na_df[na_df == 0].index).sort_values(ascending=False)
        missing_data = pd.DataFrame({'Missing Ratio %' :na_df})
        missing_data.plot(kind = "barh")
        plt.show()
    else:
        print('No NAs found')
plot_nas(df1)
plot_width, plot_height = (24,18)
plt.rcParams['figure.figsize'] = (plot_width,plot_height)
plt.rcParams['font.size']=15

Results here 


In [20]:
## Here we will check the features which have some nan values data
#features_with_na=[features for features in df.columns if df[features].isnull().sum()>1]
for feature in df1.columns:
    print('Feature: {}  Missing: {}  Categories are {}'.format(feature,np.round(df1[feature].isnull().mean(), 4),len(df1[feature].unique())))

Reuslts here 


In [66]:
import seaborn as sns
sns.heatmap(df.isnull(), cbar=False)

NameError: name 'sns' is not defined

In [35]:
# Saving the file to the datfiles raw folder
df1.to_csv('/home/workspace/files/aryan/data_files_raw/eet_neet.csv', index=False)